In [1]:
import boto.ec2
from paramiko import SSHClient
import paramiko
from termcolor import colored
import boto3
import click
import time
from s3_util import *
from ec2 import *
from spot_instance import *

In [2]:
def partitionsFinished(partitions,cache):
    partitions = partitions.split()
    partitions = [int(x) for x in partitions]
    allFinished = True
    for p in partitions:
        if p in cache:
            continue
        
        if not fileExistsS3('results/illinois-temporal/%d.ser.tgz'%p) or not fileExistsS3('results/illinois-temporal/%d.timeline.tgz'%p):
            print "partition %d unfinished" % p
            allFinished = False
            break
        else:
            print "partition %d finished" %p
            cache.add(p)
    return allFinished
def cplog2s3(ip):
    log_name = 'ip-%s.log' % ip.replace('.','-')
    command = "source ~/.customrc; cptos3.sh ~/log/illinois-temporal/%s logs/illinois-temporal/%s" % (log_name,log_name)
    run_command(host,inline=True,verbose=True,command=command)
    time.sleep(10)
def stopInstanceByIp(ip):
    conn=boto.ec2.connect_to_region('us-east-1')
    reservations=conn.get_all_instances()
    instances=[i for r in reservations for i in r.instances]
    for ins in instances:
        if ins.ip_address == ip:
            print ins.id+" is stopped now."
            boto.connect_ec2().terminate_instances(ins.id)
def splitPartitions(partitions, n):
    # split partitions into n pieces
    ret = [[]]*n
    i = 0
    for p in partitions:
        ret[i] = ret[i]+[p]
        i+=1
        if i>=n:
            i=0
    ret = [str(x).replace(',','').replace('[','').replace(']','') for x in ret]
    return ret

In [3]:
n = 2
partitions = list_unfinished_partitions()
partitions = [x for x in partitions if x > 20 and x < 25]
partitions = splitPartitions(partitions,n)

In [4]:
tag = 'Timeline'
forceUpdate = 'false'
MAX_NUM_EVENT = '100'
new_reservation = request_spot_instances(count=n,tag=tag)
print "Wait for 3 mins"
time.sleep(180)
run_command_all_instances(tag=tag,inline=False,target='./init_script.sh')
time.sleep(30)

ips=get_all_instances(tag)
ips.sort()
command = "./scripts/runIllinoisTemporal.sh "
par2host = {}
for i,host in enumerate(ips):
    print "--------------------"
    print "%d - Executing scripts on %s" %(i,host)
    par2host[partitions[i]] = host
    command_modified = command+"\"%s\" %s %s >> ~/log/illinois-temporal/ip-%s.log &" % (partitions[i], forceUpdate, MAX_NUM_EVENT, host.replace('.','-'))
    run_command(host,command_modified,inline=True,verbose=False)

All have ips assigned.
35.170.53.188 (pending) isn't running.
34.200.239.123 (pending) isn't running.
All aren't running. Wait for 10 sec...
35.170.53.188 (pending) isn't running.
34.200.239.123 (pending) isn't running.
All aren't running. Wait for 10 sec...
35.170.53.188 is running.
34.200.239.123 is running.
All are running.
35.170.53.188
34.200.239.123
Wait for 3 mins
There're 4 instances.
2 are running now.
--------------------
0 - Executing scripts on 34.200.239.123


/home/qning2/Software/anaconda2/lib/python2.7/site-packages/paramiko/client.py:822: UserWarning: Unknown ssh-ed25519 host key for 34.200.239.123: f7b8b303f555cdc9e37843ab83bd1452
  key.get_name(), hostname, hexlify(key.get_fingerprint())


Run command:#!/bin/bash
source ~/.customrc
./scripts/cpfroms3.sh scripts/runIllinoisTemporal.sh scripts/runIllinoisTemporal.sh && chmod 774 ~/scripts/runIllinoisTemporal.sh &
./scripts/cpfroms3.sh packages/illinois-temporal/illinois-temporal-1.0.0.jar ~/code/illinois-temporal/illinois-temporal-1.0.0.jar
mkdir -p data/nyt-annotated-jsons
mkdir -p log/illinois-temporal
mkdir -p results/illinois-temporal
--------------------
1 - Executing scripts on 35.170.53.188


/home/qning2/Software/anaconda2/lib/python2.7/site-packages/paramiko/client.py:822: UserWarning: Unknown ssh-ed25519 host key for 35.170.53.188: 352fd645b0f6522aa90da62176c89d22
  key.get_name(), hostname, hexlify(key.get_fingerprint())


Run command:#!/bin/bash
source ~/.customrc
./scripts/cpfroms3.sh scripts/runIllinoisTemporal.sh scripts/runIllinoisTemporal.sh && chmod 774 ~/scripts/runIllinoisTemporal.sh &
./scripts/cpfroms3.sh packages/illinois-temporal/illinois-temporal-1.0.0.jar ~/code/illinois-temporal/illinois-temporal-1.0.0.jar
mkdir -p data/nyt-annotated-jsons
mkdir -p log/illinois-temporal
mkdir -p results/illinois-temporal
There're 4 instances.
2 are running now.
--------------------
0 - Executing scripts on 34.200.239.123
Run command:./scripts/runIllinoisTemporal.sh "23" false 100>>~/log/illinois-temporal/ip-34-200-239-123.log &
--------------------
1 - Executing scripts on 35.170.53.188
Run command:./scripts/runIllinoisTemporal.sh "24" false 100>>~/log/illinois-temporal/ip-35-170-53-188.log &


In [15]:
par2host = {}
for index,par in enumerate(partitions):
    par2host[par] = ips[index]
par2host

{'21': '34.200.212.126', '22': '35.170.58.27'}

In [7]:
partitions

['23', '24']

In [8]:
partitions_remaining = partitions
cache = set()
while 1:
    for p_str in partitions_remaining:
        if partitionsFinished(p_str,cache):
            print "%s finished" % p_str
            print "Copy cmd log to s3"
            partitions.remove(p_str)
            host = par2host[p_str]
            cplog2s3(host)
            print "Stop instance"
            #stopInstanceByIp(host)
        else:
            print "Wait for 10 min"
            time.sleep(60*1)
            #time.sleep(5)
    if len(partitions)==0:
        break
    partitions_remaining = partitions

partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
partition 24 unfinished
Wait for 10 min
partition 23 unfinished
Wait for 10 min
